# RGBs and Other Composites

Meteorological satellite instrument bands provide a look at the Earth at various wavelengths. Each band represents the electro-magnetic radiation observed at one wavelength or range of wavelengths. While this is very useful, flexible, and requires a very limited amount of processing to visualize, it does require that the viewer know what to expect from each band or wavelength. In addition to viewing individual bands, another common option for viewing satellite data is to combine multiple bands in to one image such as RGB (Red, Green, Blue) images. By assigning certain bands to particular channels in the output RGB image we can provide an easy way to identify features in the data. Instead of having to know what a particular wavelength is sensitive to, we can now focus on what colors in particular RGB recipe mean.

Using our ABI CONUS data from before, let's look at some of the built-in RGB composites provided by Satpy.

In [ ]:
%run ../init_notebook.py
from satpy import Scene
from glob import glob

filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.available_composite_names()

The above list shows what composites Satpy knows how to make and that it has the inputs for. A lot of the recipes in Satpy come from standard recipes from science agencies like EUMETSAT, NOAA, or NASA. Let's first look at the `airmass` RGB. The `airmass` RGB is made of the following bands:

```
R: C08 - C10
G: C12 - C13
B: C08
```

The red channel is the difference between the C08 (6.185µm) and C10 (7.34µm) bands, the green channel is the difference between the C12 (9.61µm) and C13 (10.35µm) bands, and the blue channel is the C08 (6.185µm) band.

The EUMETSAT EUMeTrain manual summarizes the use of the `airmass` RGB as:

> As both water vapor channels (WV 6.2 µm and WV 7.3 µm) are included in this composite, the main applications are the detection of dynamic processes, such as rapid cyclogenesis, jet streams and PV anomalies.

Read more on the [EUMeTrain website](http://www.eumetrain.org/data/3/306/navmenu.php?tab=9&page=4.0.0).

In Satpy, we can load preconfigured composites like any normal file-provided product. Satpy will look at the recipe, determine what bands need to loaded, what modifications/corrections need to be applied to satisfy the recipe, and then give us the `DataArray` representing that RGB back.

In [ ]:
scn.load(['airmass'])
scn['airmass']

The key part of the above xarray output is that this is the first 3 dimensional DataArray we've worked with in Satpy with the `(bands: 3, y: 1500, x: 2500)` dimensions. The `bands` dimension tells us what each "band" represents. In this case Red, Green, and Blue channels of an RGB image.

In [ ]:
scn['airmass'].coords['bands']

Let's save this product to a PNG image to get an idea of what an `airmass` RGB looks like.

In [ ]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    scn.save_datasets(writer='simple_image')

We can see in the image how the various bands combine in different ways and certain colors can represent specific states of the Earth.

Earlier in this tutorial we talked about "enhancing" data; the process of scaling data from observed scientific data values to image values (ex. 0-255 integers). This process was automatically done for us by the `save_datasets`. If we want to display our data in a matplotlib figure we need to do this manually. Satpy provides a `get_enhanced_image` utility function to simplify this.

We can tell xarray what dimension of our `DataArray` represents the "RGB" axis. We do this by passing the `rgb` keyword argument with the name of the dimension. In Satpy this dimension should always be called `'bands'`. Without this matplotlib would not be able to plot our RGB properly.

Lastly, we'll tell the `imshow` method that we *know* that our RGB will be between 0 and 1 with the `vmin` and `vmax` keyword arguments so there is no need to compute the min/max of the data.

Let's plot our `airmass` RGB composite.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from satpy.writers import get_enhanced_image

plt.figure()
img = get_enhanced_image(scn['airmass'])
# get DataArray out of `XRImage` object
img_data = img.data
img_data.plot.imshow(rgb='bands', vmin=0, vmax=1)

## Compositing Different Resolutions

When generating the `airmass` RGB we were fortunate in that all of the input data was the same resolution. Let's look at the `natural_color` RGB now, where the input resolutions differ. The `natural_color` RGB is useful as described by EUMeTrain for seeing snow and ice:

> Snow on the ground as well as ice over mountains, frozen lakes and sea ice appear cyan in the Natural Colour RGB images.

See the [EUMeTrain website](http://www.eumetrain.org/resources/MSG_natcolour_cyan_snow.html) for more information.

For the ABI instrument this composite is a combination of the following channels:

```
R: C05 (1000m)
G: C03 (1000m)
B: C02 (500m)
```

This means that we are trying to combine arrays of different shapes. The only way we know to make data the same resolution and make it easier to compare or combine, is to resample. Let's load this composite and see how Satpy handles generating a composite like this.

In [ ]:
scn.load(['natural_color'])

In [ ]:
'natural_color' in scn

In [ ]:
scn.missing_datasets

We didn't do anything wrong, but Satpy doesn't have a `natural_color` composite in the `Scene`. Satpy knows that the reason it wasn't able to generate this composite was because the data was not all the same resolution. The `Scene` knows that we need to resample before we can be able to generate this composite. Let's do that now by using the `native` resampler. We'll also use the `min_area` method to aggregate the data to the lowest resolution in the Scene; 2km in this case (from the `airmass` dependencies):

In [ ]:
new_scn = scn.resample(scn.min_area(), resampler='native')
new_scn['natural_color']

 Once resampling had completed, the `Scene` attempted to regenerate any composites that were previous requested. Once successfully generated the `natural_color` RGB shows up like any other product. Let's follow the same procedure we used for plotting the `airmass` RGB, but for the `natural_color` RGB. Note we are using the `new_scn` because the `natural_color` RGB only exists in this resampled `Scene`.

In [ ]:
with ProgressBar():
    plt.figure()
    img = get_enhanced_image(new_scn['natural_color'])
    # get DataArray out of `XRImage` object
    img_data = img.data
    img_data.plot.imshow(vmin=0, vmax=1, rgb='bands')

## Custom RGBs

Now we've learned how to look at the available builtin composites, load them, and generate them. Let's make our own custom RGB. Let's recreate our Scene from before and load three channels that we'd like to put in an RGB.

In [ ]:
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.available_dataset_names()

In [ ]:
r_band = 'EDITME'
g_band = 'EDITME'
b_band = 'EDITME'
scn.load([r_band, g_band, b_band])

Our `Scene` object now has three channels loaded. We'll now use the special `GenericCompositor` from Satpy's builtin composites to join these arrays together for us and make an RGB. Let's see what happens when we attempt this:

In [ ]:
from satpy.composites import GenericCompositor
comp = GenericCompositor('my_rgb')
my_rgb = comp((scn[r_band], scn[g_band], scn[b_band]))

If the channels you choose are different resolutions, you should have gotten an `IncompatibleAreas` exception from the above operation. Just like with `natural_color` before, this is Satpy's compositors telling us that our inputs are at different resolutions and they need to be resampled.

Let's resample using the `native` resampler to fix the resolutions and try regenerating our composite with the newly resampled `Scene`.

Do the below step even if you didn't receive an error. It isn't necessary, but will make the remaining instructions easier.

In [ ]:
new_scn = scn.resample(resampler='native')
my_rgb = comp((new_scn[r_band], new_scn[g_band], new_scn[b_band]))
my_rgb

We successfully generated an RGB `DataArray` object. Let's plot it and see what it looks like.

In [ ]:
plt.figure()
img = get_enhanced_image(my_rgb)
# get DataArray out of `XRImage` object
img_data = img.data
img_data.plot.imshow(vmin=0, vmax=1, rgb='bands')

The above image probably looks very white. This is because Satpy assumes that RGBs, unless configured otherwise, should be scaled between 0 and 1, but our data hasn't been **normalized** yet. We told xarray and matplotlib that our RGB's data was between 0 and 1, but depending on the channels you chose it was actually much higher than that (ex. 0-120% for reflectance, 200-250K for brightness temperatures).

### Exercise:

**Time: 10-15 minutes**

To find the ranges that make our data look the best, let's linearly scale our data below. We'll scale each band separately where the first number represents our minimum, the second our maximum, and the third is our minimum again. The end result should be an RGB `DataArray` where "main" part of our data is between 0 and 1. Keep in mind what type of data we are scaling; reflectances or brightness temperatures.

Remember the limits that produce the best image for you. We'll use them in the next section.

In [ ]:
r_norm = (new_scn[r_band] - 220) / (260 - 220)
g_norm = (new_scn[g_band] - 220) / (260 - 220)
b_norm = (new_scn[b_band] - 0) / (120 - 0)
my_rgb = comp((r_norm, g_norm, b_norm))

plt.figure()
my_rgb.plot.imshow(vmin=0, vmax=1, rgb='bands')

## Configure Composite Recipes

Now that we've found an RGB recipe that we like, we may want to reuse it later. We could write a python script or module that generates this, but that requires that we know what bands go in to the RGB when loading from the Scene and can make using the composite more difficult.

Satpy was originally built to help automatic and repeated operational processing. To help with this Satpy uses a series of YAML configuration files that are builtin to the package, but can also be customized by users. This tutorial comes with skeleton YAML files to be filled in for this exercise that we will fill in now.

1. Add the composite recipe

   To add the band combinations to a YAML configuration, first navigate to the `notebooks/composites/` directory [here](/tree/notebooks/composites) and open the `abi.yaml` file in a text editor. Or you can use Jupyter Notebook's editor by going to [abi.yaml](/edit/notebooks/composites/abi.yaml). We want to add the below text to this file:
   
   ```yaml
   composites:
     my_rgb:
       compositor: !!python/name:satpy.composites.GenericCompositor
       prerequisites:
       - name: YOUR_R_CHANNEL
       - name: YOUR_G_CHANNEL
       - name: YOUR_B_CHANNEL
    ```
    
    Since this `composites` directory is in our current directory Satpy will find it and include our composite as something we can load. We've told it our composite's name is `my_rgb`, that we want to use the `GenericCompositor` class, and that it depends on the three channels we've specified.
    
2. Add the enhancement recipe

   To add our color limits we will edit a separate file in `notebooks/enhancements/abi.yaml`. Click [here](/edit/notebooks/enhancements/abi.yaml) to edit the file using Jupyter Notebook's editor. We want our enhancement to look like this:
   
   ```yaml
   enhancements:
     my_rgb:
       name: my_rgb
       operations:
       - name: color_limits
         method: !!python/name:satpy.enhancements.stretch
         kwargs: {stretch: 'crude', min_stretch: [RMIN, GMIN, BMIN], max_stretch: [RMAX, GMAX, BMAX]}
   ```
   
   This will use the builtin `stretch` method to do a simple linear scaling between our specified minimum and maximum values. You should replace the `XMIN` and `XMAX` values with the values you came up with in the above exercise.
   

Now that we've configured our composite for creation and scaling, we can load it like any other composite in Satpy. Note how we don't have to explicitly preload our input channels, our recipe tells the `Scene` what it needs.

In [ ]:
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.available_composite_names()

In [ ]:
scn.load(['my_rgb'])

Just like before, if our composite needs multiple resolutions then we need to resample to make all inputs the same size.

In [ ]:
new_scn = scn.resample(resampler='native')
new_scn['my_rgb']

We successfully generated our RGB, let's plot it. We'll use the `get_enhanced_image` function like we did at the beginning of this lesson to apply our configured enhancement.

In [ ]:
plt.figure()
# get DataArray out of `XRImage` object with .data
img_data = get_enhanced_image(new_scn['my_rgb']).data
img_data.plot.imshow(vmin=0, vmax=1, rgb='bands')

Now we've loaded our composite from a pre-configured YAML file and displayed it just like the `airmass` and `natural_color` RGB we started with. We could have used other methods we learned about earlier, like `crop` or `save_datasets`, to do other operations. We'll keep using this RGB in the next couple lessons and continue to show how reusing a configured recipe can be make our work easier.